In [1]:
# import sys
# import gymnasium as gym
# sys.modules["gym"] = gym
import gym
gym.__version__

'0.21.0'

In [2]:
import gnwrapper

In [3]:
import os
import numpy as np
import torch
print(torch.cuda.is_available())

True


In [4]:
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy

In [5]:
env = gym.make("CartPole-v1")

In [6]:
expert = PPO(
    policy=MlpPolicy,
    env=env,
    seed=0,
    batch_size=64,
    ent_coef=0.0,
    learning_rate=0.0003,
    n_epochs=10,
    n_steps=64,
)
expert.learn(1_000_000, progress_bar=True)
expert.save("ppo_CartPole_expert")

 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 999,809/1,000,000  [ 0:10:16 < 0:00:01 , 1,817 it/s ]
 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 999,809/1,000,000  [ 0:10:16 < 0:00:01 , 1,817 it/s ]

In [7]:
expert = PPO.load("ppo_CartPole_expert.zip", print_system_info=True)

== CURRENT SYSTEM INFO ==
- OS: Linux-5.15.0-58-generic-x86_64-with-glibc2.35 # 64-Ubuntu SMP Thu Jan 5 11:43:13 UTC 2023
- Python: 3.10.6
- Stable-Baselines3: 1.7.0
- PyTorch: 1.13.1+cu117
- GPU Enabled: True
- Numpy: 1.24.1
- Gym: 0.21.0

== SAVED MODEL SYSTEM INFO ==
- OS: Linux-5.15.0-58-generic-x86_64-with-glibc2.35 # 64-Ubuntu SMP Thu Jan 5 11:43:13 UTC 2023
- Python: 3.10.6
- Stable-Baselines3: 1.7.0
- PyTorch: 1.13.1+cu117
- GPU Enabled: True
- Numpy: 1.24.1
- Gym: 0.21.0



In [8]:
from stable_baselines3.common.evaluation import evaluate_policy

reward, _ = evaluate_policy(expert, env, 10)
print(reward)

/home/vlad1slav/Diploma/rl-venv/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


500.0


In [9]:
from imitation.data import rollout
from imitation.data.wrappers import RolloutInfoWrapper
from stable_baselines3.common.vec_env import DummyVecEnv
import numpy as np

rng = np.random.default_rng()
rollouts = rollout.rollout(
    expert,
    DummyVecEnv([lambda: RolloutInfoWrapper(env)]),
    rollout.make_sample_until(min_timesteps=None, min_episodes=50),
    rng=rng,
)
transitions = rollout.flatten_trajectories(rollouts)

In [10]:
print(
    f"""The `rollout` function generated a list of {len(rollouts)} {type(rollouts[0])}.
After flattening, this list is turned into a {type(transitions)} object containing {len(transitions)} transitions.
The transitions object contains arrays for: {', '.join(transitions.__dict__.keys())}."
"""
)

The `rollout` function generated a list of 50 <class 'imitation.data.types.TrajectoryWithRew'>.
After flattening, this list is turned into a <class 'imitation.data.types.Transitions'> object containing 25000 transitions.
The transitions object contains arrays for: obs, acts, infos, next_obs, dones."



In [12]:
from imitation.algorithms import bc

bc_trainer = bc.BC(
    observation_space=env.observation_space,
    action_space=env.action_space,
    demonstrations=transitions,
    rng=rng,
)

In [13]:
reward_before_training, _ = evaluate_policy(bc_trainer.policy, env, 10)
print(f"Reward before training: {reward_before_training}")

Reward before training: 9.5


In [16]:
bc_trainer.train(n_epochs=2, progress_bar=True)
reward_after_training, _ = evaluate_policy(bc_trainer.policy, env, 10)
print(f"Reward after training: {reward_after_training}")


0batch [00:00, ?batch/s]

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 0         |
|    ent_loss       | -0.000158 |
|    entropy        | 0.158     |
|    epoch          | 0         |
|    l2_loss        | 0         |
|    l2_norm        | 183       |
|    loss           | 0.118     |
|    neglogp        | 0.118     |
|    prob_true_act  | 0.917     |
|    samples_so_far | 32        |
---------------------------------


452batch [00:00, 762.80batch/s]

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 500       |
|    ent_loss       | -0.000152 |
|    entropy        | 0.152     |
|    epoch          | 0         |
|    l2_loss        | 0         |
|    l2_norm        | 184       |
|    loss           | 0.125     |
|    neglogp        | 0.125     |
|    prob_true_act  | 0.922     |
|    samples_so_far | 16032     |
---------------------------------


769batch [00:01, 777.50batch/s]
926batch [00:01, 762.24batch/s]

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 1000      |
|    ent_loss       | -0.000244 |
|    entropy        | 0.244     |
|    epoch          | 1         |
|    l2_loss        | 0         |
|    l2_norm        | 185       |
|    loss           | 0.229     |
|    neglogp        | 0.229     |
|    prob_true_act  | 0.84      |
|    samples_so_far | 32032     |
---------------------------------


1481batch [00:01, 781.92batch/s]

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 1500      |
|    ent_loss       | -0.000173 |
|    entropy        | 0.173     |
|    epoch          | 1         |
|    l2_loss        | 0         |
|    l2_norm        | 187       |
|    loss           | 0.144     |
|    neglogp        | 0.144     |
|    prob_true_act  | 0.898     |
|    samples_so_far | 48032     |
---------------------------------


1560batch [00:02, 783.75batch/s]
1562batch [00:02, 765.12batch/s]


Reward after training: 500.0
